# Fatkun 활용 이미지 분석
- 쿠팡에서 셔츠와 청바지의 이미지를 일괄 다운로드 했다.
- 셔츠와 청바지를 jeans_shirts.zip파일로 압축 후 파일에 업로드
- ImageDataGenerator를 통해 셔츠와 청바지를 분류해보자.

## 데이터 가져오기


In [1]:
# 압축 풀기
import os
import zipfile

# 압축 해제할 폴더 지정
extract_path = '/content/jeans_shirts'

local_zip = '/content/jeans_shirts.zip'
# 압축 해제
with zipfile.ZipFile(local_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [5]:
# 데이터 경로: extract_path

# 청바지, 셔츠 경로
jeans_dir = os.path.join(extract_path, 'jeans') # /content/jeans_shirts/jeans
shirts_dir = os.path.join(extract_path, 'shirts')  # /content/jeans_shirts/shirts

In [6]:
print('청바지 사진 수 : ', len(os.listdir(jeans_dir)))
print('셔츠 사진 수 : ', len(os.listdir(shirts_dir)))

청바지 사진 수 :  72
셔츠 사진 수 :  76


In [17]:
from PIL import Image
import numpy as np

# 이미지 전처리 함수
def load_and_preprocess_image(file_path, target_size=(64, 64)):
    # 이미지 열기
    img = Image.open(file_path)

    # 이미지 크기 조절
    img = img.resize(target_size)

    # 이미지를 NumPy 배열로 변환 (0에서 255 사이의 값)
    img_array = np.array(img)

    # 모델에 맞는 입력 형태로 변환 (예: CNN 모델을 위해 차원 추가)
    img_array = np.expand_dims(img_array, axis=0)

    # 모델에 맞는 전처리 (예: 이미지 픽셀 값을 0에서 1로 정규화)
    img_array = img_array.astype('float32') / 255.0

    return img_array

In [19]:
import os
import shutil
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

# 늘린 데이터를 저장할 폴더 지정
original_data_dir = '/content/jeans_shirts'
many_data_dir = '/content/many_jeans_shirts'

# 스케일링 + ImageDataGenerator 속성 설정
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest'
)

# 각 클래스(셔츠, 청바지)의 데이터 개수를 지정
num_jeans = 72
num_shirts = 76

# 각 클래스의 데이터를 늘리고 지정된 폴더에 저장
for class_name, num_images in [('jeans', num_jeans), ('shirts', num_shirts)]:
    class_original_dir = os.path.join(original_data_dir, class_name)
    class_many_dir = os.path.join(many_data_dir, class_name)

    # 새로운 폴더 생성
    os.makedirs(class_many_dir, exist_ok=True)

    # 데이터 증강 수행
    for img_file in os.listdir(class_original_dir)[:num_images]:
        img_path = os.path.join(class_original_dir, img_file)
        img = load_and_preprocess_image(img_path, target_size=(64, 64))  # 이미지를 불러와서 필요한 전처리 수행

        # 이미지를 늘리고 저장
        i = 0
        for batch in datagen.flow(img, save_to_dir=class_many_dir, save_prefix='many', save_format='rgba'):
            i += 1
            if i >= 20:  # 하나의 이미지를 10번 늘림
                break

In [20]:
print(dataGen.class_indices)

{'jeans': 0, 'shirts': 1}


In [21]:
print('증식된 청바지 사진 수 : ', len(os.listdir('/content/many_jeans_shirts/jeans')))
print('증식된 셔츠 사진 수 : ', len(os.listdir('/content/many_jeans_shirts/shirts')))

증식된 청바지 사진 수 :  2689
증식된 셔츠 사진 수 :  1891


In [22]:
# 데이터를 훈련 및 검증 데이터로 나누기
train_data_dir = '/content/train_data'
val_data_dir = '/content/val_data'
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(val_data_dir, exist_ok=True)

for class_name in ['shirts', 'jeans']:
    class_many_dir = os.path.join(many_data_dir, class_name)
    class_train_dir = os.path.join(train_data_dir, class_name)
    class_val_dir = os.path.join(val_data_dir, class_name)

    # 새로운 폴더 생성
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_val_dir, exist_ok=True)

    # 데이터를 훈련 및 검증 데이터로 나눔
    all_files = os.listdir(class_many_dir)
    train_files, val_files = train_test_split(all_files, test_size=0.2, random_state=42)

    # 파일을 각각의 폴더로 복사
    for file in train_files:
        shutil.copy(os.path.join(class_many_dir, file), os.path.join(class_train_dir, file))

    for file in val_files:
        shutil.copy(os.path.join(class_many_dir, file), os.path.join(class_val_dir, file))

In [23]:
# 훈련셋 중에 청바지 사진 경로
train_jeans_dir = os.path.join(train_data_dir, 'jeans')
# 훈련셋 중에 셔츠 사진 경로
train_shirts_dir = os.path.join(train_data_dir, 'shirts')

# 검증셋 중에 청바지 사진 경로
validation_jeans_dir = os.path.join(val_data_dir, 'jeans')
# 검증셋 중에 셔츠 사진 경로
validation_shirts_dir = os.path.join(val_data_dir, 'shirts')

In [24]:
print('훈련 데이터 중 청바지 사진 수 : ', len(os.listdir(train_jeans_dir)))
print('훈련 데이터 중 셔츠 사진 수 : ', len(os.listdir(train_shirts_dir)))

print('검증 데이터 중 청바지 사진 수 : ', len(os.listdir(validation_jeans_dir)))
print('검증 데이터 중 셔츠 사진 수 : ', len(os.listdir(validation_shirts_dir)))

훈련 데이터 중 청바지 사진 수 :  2151
훈련 데이터 중 셔츠 사진 수 :  1512
검증 데이터 중 청바지 사진 수 :  538
검증 데이터 중 셔츠 사진 수 :  379


In [ ]:
# 한 번 더 증식

# 스케일링 + 데이터 증식 속성 설정
datagen = ImageDataGenerator(
                            horizontal_flip = True,
                            vertical_flip = True,
                            shear_range = 0.5,
                            brightness_range = [0.5, 1.5],
                            zoom_range = 0.2,
                            width_shift_range = 0.1,
                            height_shift_range = 0.1,
                            rotation_range = 30,
                            fill_mode = 'nearest')

# 데이터 스케일링 및 증식
trainGen = datagen.flow_from_directory(
    os.path.join(rootPath, 'train'),
    target_size=(64, 64),
    class_mode = 'binary'
)

testGen = testImageGenerator.flow_from_directory(
    os.path.join(rootPath, 'validation'),
    target_size=(64, 64),
    class_mode = 'binary'
)